<a href="https://colab.research.google.com/github/JRMJ14/JRMJ14/blob/main/clima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 12.0 MB/s eta 0:00:00


**Rutas de navegacion**

juego donde puedes implementar un entorno que simule diferentes condiciones climáticas donde el agente deba tomar decisiones sobre cómo navegar, por ejemplo, rutas de navegación por aire o mar que eviten condiciones peligrosas.

In [2]:
#importamos las respectivas librerias necesarias para el desarrollo del entorno
import gymnasium as gym
import numpy as np
import random

In [3]:
class NavigationEnv(gym.Env):
    def __init__(self):
        super(NavigationEnv, self).__init__()

        # Tamaño del entorno de navegación (10x10)
        self.grid_size = 10

        # Espacio de observación: todas las posiciones posibles (10x10 = 100 estados)
        self.observation_space = gym.spaces.Discrete(self.grid_size ** 2)

        # Espacio de acción: arriba, abajo, izquierda, derecha
        self.action_space = gym.spaces.Discrete(4)

        # Definir punto de inicio y destino
        self.start_position = (0, 0)
        self.goal_position = (9, 9)

        # Zonas de clima adverso (ubicaciones peligrosas)
        self.bad_weather_zones = [(3, 3), (4, 5), (6, 6), (7, 2), (8, 8)]

        # Estado inicial del agente
        self.state = self.start_position

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.state = (random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1))  # Reiniciar el agente al inicio
        #self.state = self.start_position  # Reiniciar el agente al inicio
        return self._get_state_index(self.state), {}

    def step(self, action):
        x, y = self.state

        # Movimiento basado en la acción seleccionada
        if action == 0 and y > 0:          # Arriba
            y -= 1
        elif action == 1 and y < self.grid_size - 1:  # Abajo
            y += 1
        elif action == 2 and x > 0:        # Izquierda
            x -= 1
        elif action == 3 and x < self.grid_size - 1:  # Derecha
            x += 1

        # Actualizar el estado del agente
        self.state = (x, y)

        # Recompensas y finalización
        reward = -1  # Penalización por cada paso
        done = False

        # Si alcanza el destino, gran recompensa
        if self.state == self.goal_position:
            reward = 20
            done = True
        # Penalización adicional por entrar en clima peligroso
        elif self.state in self.bad_weather_zones:
            reward = -10

        return self._get_state_index(self.state), reward, done, False, {}

    def _get_state_index(self, position):
        """Convierte una posición (x, y) a un índice único para la Q-table."""
        return position[0] * self.grid_size + position[1]

In [5]:
# Crear el entorno
env = NavigationEnv()

In [6]:
# Parámetros de aprendizaje Q-Learning
q_table = np.zeros([env.observation_space.n, env.action_space.n])  # Tabla Q inicial
alpha = 0.1       # Tasa de aprendizaje
gamma = 0.9       # Factor de descuento
epsilon = 1.0     # Factor de exploración inicial
epsilon_decay = 0.800  # Decaimiento de epsilon para reducir la exploración
min_epsilon = 0.01

In [7]:
# Entrenamiento del agente
num_episodes = 1000
for episode in range(num_episodes):
    state, _ = env.reset()
    done = False

    while not done:
        # Elegir acción (exploración vs explotación)
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Exploración
        else:
            action = np.argmax(q_table[state])  # Explotación

        # Ejecutar la acción en el entorno
        next_state, reward, done, _, _ = env.step(action)

        # Actualizar la Q-table
        best_next_action = np.argmax(q_table[next_state])
        td_target = reward + gamma * q_table[next_state, best_next_action]
        td_error = td_target - q_table[state, action]
        q_table[state, action] += alpha * td_error

        # Avanzar al siguiente estado
        state = next_state

    # Decaer epsilon después de cada episodio
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

print("Entrenamiento completado.")

Entrenamiento completado.


In [8]:
# Evaluación del agente
test_episodes = 10
for episode in range(test_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = np.argmax(q_table[state])  # Elegir la mejor acción
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        state = next_state
    print(f"Recompensa total en episodio de prueba {episode + 1}: {total_reward}")

Recompensa total en episodio de prueba 1: 11
Recompensa total en episodio de prueba 2: 9
Recompensa total en episodio de prueba 3: 12
Recompensa total en episodio de prueba 4: 13
Recompensa total en episodio de prueba 5: 13
Recompensa total en episodio de prueba 6: 19
Recompensa total en episodio de prueba 7: 15
Recompensa total en episodio de prueba 8: 18
Recompensa total en episodio de prueba 9: 17
Recompensa total en episodio de prueba 10: 6


In [9]:
print("Tabla Q después del entrenamiento:")
print(q_table)

Tabla Q después del entrenamiento:
[[-4.61504326e+00 -4.62458195e+00 -4.62682738e+00 -4.63551231e+00]
 [-4.57771535e+00 -4.51647896e+00 -4.53693923e+00 -4.52611073e+00]
 [-4.37659201e+00 -4.35425810e+00 -4.33089925e+00 -4.33054615e+00]
 [-4.16804382e+00 -4.14095654e+00 -4.12533197e+00 -3.98428916e+00]
 [-3.88534746e+00 -3.84682325e+00 -3.88321588e+00 -3.67921097e+00]
 [-3.56211610e+00 -2.90512084e+00 -3.55959188e+00 -3.51728190e+00]
 [-3.24227722e+00 -3.19864911e+00 -3.20863167e+00 -6.42985947e-01]
 [-2.96588055e+00 -2.94368152e+00 -2.96537260e+00 -2.83686957e+00]
 [-2.75149307e+00 -2.62522624e+00 -2.80523290e+00 -2.74141369e+00]
 [-2.58594009e+00 -2.58343578e+00 -2.60073021e+00 -7.45476567e-01]
 [-4.51943816e+00 -4.52752717e+00 -4.53254927e+00 -4.55590326e+00]
 [-4.44310272e+00 -4.42293050e+00 -4.43923771e+00 -4.43596524e+00]
 [-4.19735312e+00 -3.77527853e+00 -4.18989880e+00 -4.18515436e+00]
 [-3.90789486e+00 -2.05890900e+00 -3.95053631e+00 -3.90854465e+00]
 [-3.62260176e+00 -3.609686